<a href="https://colab.research.google.com/github/neerajgosai19/data-science/blob/master/Speech_Services.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ibm_watson bs4

     |████████████████████████████████| 389kB 5.2MB/s 
     |████████████████████████████████| 204kB 11.8MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-4.7.1-cp36-none-any.whl size=379133 sha256=73cfd03e97cad1751f5bb8d22afec3be597d17d8c5e43bbdad27f43616bc296a
  Stored in directory: /root/.cache/pip/wheels/6e/14/69/dbbd573a3bab3bf64984572284f13f174f430038308abdd73c
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.7.3-cp36-none-any.whl size=45945 sha256=f368ebdc359b992d0d817cf2ffdde00d29d9e34650257c7c80a6e04ff813adc9
  Stored in directory: /root/.cache/pip/wheels/34/6e/58/589e0f841c2fae9dad99630d78ddc7a60c5c7663a16a39cdbb
Successfully built ibm-watson ibm-cloud-sdk-core


In [ ]:
import os
from glob import glob
from bs4 import BeautifulSoup
import IPython
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import SpeechToTextV1
from ibm_watson import AssistantV2
from ibm_watson import TextToSpeechV1

In [ ]:
recognition_service = SpeechToTextV1(IAMAuthenticator('INogA2g1OymnqMaYUq0nKdc9F8Oqrp820DoNh-xl85u9'))
recognition_service.set_service_url('https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/59d79e72-ded7-484b-b1ce-e7e1909ebc91')
SPEECH_EXTENSION = "*.webm"
SPEECH_AUDIOTYPE = "audio/webm"

def recognize_audio():
    while len(glob(SPEECH_EXTENSION)) == 0:
        pass
    filename = glob(SPEECH_EXTENSION)[0]
    audio_file = open(filename, "rb")
    os.popen("rm " + filename)
    result = recognition_service.recognize(audio=audio_file, content_type=SPEECH_AUDIOTYPE).get_result()
    return result["results"][0]["alternatives"][0]["transcript"]

In [ ]:
assistant = AssistantV2(version='2019-02-28', authenticator=IAMAuthenticator('Qs8c8t_PoYLHQw0jtmpvDOIf5OUejN2_ezJELPdPo7pQ'))
assistant.set_service_url('https://api.kr-seo.assistant.watson.cloud.ibm.com/instances/7614d6b5-016f-459d-a510-f936c9ea2876')
ASSISTANT_ID = "b63d8d8b-039c-40e5-ba32-fc2c5fd511be"
session_id = assistant.create_session(assistant_id =ASSISTANT_ID).get_result()["session_id"]

def message_assistant(text):
    response = assistant.message(assistant_id=ASSISTANT_ID,
                                 session_id=session_id,
                                 input={'message_type': 'text', 'text': text}).get_result()
    return BeautifulSoup(response["output"]["generic"][0]["text"]).get_text()

In [ ]:
synthesis_service = TextToSpeechV1(IAMAuthenticator('LMPxVTQxi4xjeIEsXNFj7KDdQ94l2gIr34l2hRzGwPbl'))
synthesis_service.set_service_url('https://api.eu-gb.text-to-speech.watson.cloud.ibm.com/instances/bf685348-158e-4f8d-a8cd-efce25ea8f58')

def speak_text(text):
    with open('temp.wav', 'wb') as audio_file:
        response = synthesis_service.synthesize(text, accept='audio/wav', voice="en-US_AllisonV3Voice").get_result()
        audio_file.write(response.content)
    return IPython.display.Audio("temp.wav", autoplay=True)

In [ ]:
speak_text(message_assistant(recognize_audio()))